In [ ]:
!pip install yfinance

In [ ]:
import yfinance
import numpy
import pandas

from google.colab import files
from google.colab import drive
import os


In [ ]:
'''
Task Outline

a. Obtain CSV of all stock tickers ✅
b. Conduct Analysis for Bollinger Bands and Keltner Channels in mechanism for detecting consolidation ✅
c. Analysis for the Zeta Model
d. conglomerate them all into one stock screener than prints out the list of all the stock tickers that we should run Black Scholes and Monte Carlo on

'''

In [ ]:
# PART A

data = pandas.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
data[0].head()
data = data[0]['Symbol']

data.to_csv('S&P500tickers.csv', mode = 'w', index = False)
files.download('S&P500tickers.csv')

In [ ]:
with open('S&P500tickers.csv') as f:
  symbols = f.read().splitlines()
  symbols.pop(0)


  for i in symbols:
      ticker_dta = yfinance.download(i, start = "2022-04-02", end = "2023-02-11") # arbitrarily chosen 45 weeks (6 -> 65 weeks)
      drive.mount('/drive')
      ticker_dta.to_csv('/drive/My Drive/stock_screener/{}.csv'.format(i), encoding='utf-8', index=False)
  

In [ ]:
from IPython.lib.security import passwd
drive.mount('/drive')
dir = '/drive/MyDrive/stock_screener/'

def s_mov_avg(df, lookback_window):
  sma = df['Close'].rolling(window = lookback_window, min_periods = 1).mean()  
  std = df['Close'].rolling(window = lookback_window, min_periods = 1).std()
  return sma, std

def atr(df, lookback_window):
  atr_ = df['True Range'].rolling(window = lookback_window, min_periods = 1).mean()
  return atr_


def in_consolidation(curr_df):
  return (curr_df['Lower Bollinger Band'] > curr_df['Lower Keltner Channel']) \
          and (curr_df['Upper Bollinger Band']  < curr_df['Upper Keltner Channel'])

symbols_of_companies = []
for file in os.listdir(dir):
  symbol = file.split()[0]
  symbols_of_companies.append(symbol[:-4])
  curr_df = pandas.read_csv(dir + "{}".format(file))
  if not curr_df.empty:
    sma, std = s_mov_avg(curr_df, 20)
    curr_df['Upper Bollinger Band'] = sma + 2 * std
    curr_df['Lower Bollinger Band'] = sma - 2 * std
    curr_df['Middle Bollinger Band'] = sma
    curr_df['Standard Deviation'] = std

    curr_df["True Range"] = abs(curr_df["High"] - curr_df["Low"])
    curr_df['Average True Range'] = atr(curr_df, 20)

    curr_df['Upper Keltner Channel'] = curr_df['Middle Bollinger Band'] + 1.2 * curr_df['Average True Range']
    curr_df['Lower Keltner Channel'] = curr_df['Middle Bollinger Band'] - 1.2 * curr_df['Average True Range'] 

    # in consolidation

    curr_df['Consolidation Status'] = curr_df.apply(in_consolidation, axis = 1)


In [ ]:
symbols_of_companies
consolidated_status = curr_df['Consolidation Status'].to_list()

mapped_to_consol = dict(zip(symbols_of_companies, consolidated_status))

tb_deleted = []
for k,v in mapped_to_consol.items():
  if not v:
    tb_deleted.append(k)

for i in tb_deleted:
  del mapped_to_consol[i]

